[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mitiau/DNABERT-Z/blob/main/ZDNA-prediction.ipynb)

# Install dependecies and define helper functions

In [1]:
!pip install transformers
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.0 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
from google.colab import drive, files
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

In [3]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

# Select model and parameters

In [4]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [5]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'


In [6]:
!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

Downloading...
From (original): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx
From (redirected): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx&confirm=t&uuid=15909445-e7bc-418a-b591-fc1751d12bc6
To: /content/pytorch_model.bin
100% 354M/354M [00:02<00:00, 128MB/s]
Downloading...
From: https://drive.google.com/uc?id=10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
To: /content/config.json
100% 634/634 [00:00<00:00, 2.81MB/s]
Downloading...
From: https://drive.google.com/uc?id=16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
To: /content/special_tokens_map.json
100% 112/112 [00:00<00:00, 514kB/s]
Downloading...
From: https://drive.google.com/uc?id=1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
To: /content/tokenizer_config.json
100% 40.0/40.0 [00:00<00:00, 203kB/s]
Downloading...
From: https://drive.google.com/uc?id=1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0
To: /content/vocab.txt
100% 28.7k/28.7k [00:00<00:00, 75.0MB/s]


In [7]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Upload fasta files for prediction

In [8]:
!wget -O 'genome.fna.gz' https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/036/169/595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.fna.gz

--2025-06-15 12:08:51--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/036/169/595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.7, 130.14.250.10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9089934 (8.7M) [application/x-gzip]
Saving to: ‘genome.fna.gz’

genome.fna.gz       100%[===================>]   8.67M  11.7MB/s    in 0.7s    

2025-06-15 12:08:52 (11.7 MB/s) - ‘genome.fna.gz’ saved [9089934/9089934]



In [9]:
!gzip -rd 'genome.fna.gz'

# Predict and save raw outputs

In [10]:
out = []
input_file ='genome.fna'
result_dict = {}

for seq_record in SeqIO.parse(input_file, 'fasta'):
    kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
    seq_pieces = split_seq(kmer_seq)
    print(seq_record.name)
    with torch.no_grad():
        preds = []
        for seq_piece in tqdm(seq_pieces):
            input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
            outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1],axis = -1)[0,:,1]
            preds.append(outputs.cpu().numpy())
    result_dict[seq_record.name] = stitch_np_seq(preds)


    labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
    for label in range(1, max_label+1):
        candidate = np.where(labeled == label)[0]
        candidate_length = candidate.shape[0]
        if candidate_length>minimum_sequence_length:
            print(candidate[0], candidate[-1])
            out.append((seq_record.name, candidate[0], candidate[-1]))

CM069765.1


100%|██████████| 18978/18978 [11:36<00:00, 27.23it/s]


4701041 4701052
5631133 5631145
8876064 8876075
CM069766.1


100%|██████████| 10243/10243 [06:10<00:00, 27.65it/s]


5076796 5076807
CM069767.1


...

100%|██████████| 72/72 [00:02<00:00, 26.25it/s]


20139 20152
23534 23551
JAVFKY010000012.1


100%|██████████| 48/48 [00:01<00:00, 24.79it/s]


CM069771.1


100%|██████████| 110/110 [00:04<00:00, 25.52it/s]


In [12]:
out

[('CM069765.1', np.int64(4701041), np.int64(4701052)),
 ('CM069765.1', np.int64(5631133), np.int64(5631145)),
 ('CM069765.1', np.int64(8876064), np.int64(8876075)),
 ('CM069766.1', np.int64(5076796), np.int64(5076807)),
 ('CM069767.1', np.int64(4765477), np.int64(4765490)),
 ('JAVFKY010000011.1', np.int64(20139), np.int64(20152)),
 ('JAVFKY010000011.1', np.int64(23534), np.int64(23551))]

In [13]:
with open('ZDNABERT.bed', 'a') as f:
    for record in out:
        f.write(record[0] + "\t" + str(record[1]) + "\t" + str(record[2]) + "\n")

In [14]:
!head ZDNABERT.bed

CM069765.1	4701041	4701052
CM069765.1	5631133	5631145
CM069765.1	8876064	8876075
CM069766.1	5076796	5076807
CM069767.1	4765477	4765490
JAVFKY010000011.1	20139	20152
JAVFKY010000011.1	23534	23551


In [15]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 563 kB of archives.
After this operation, 1,548 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 bedtools amd64 2.30.0+dfsg-2ubuntu0.1 [563 kB]
Fetched 563 kB in 1s (862 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../bedtools_2.30.0+dfsg-2ubuntu0.1_amd64.deb ...
Unpacking bedtools (2.30.0+dfsg-2ubuntu0.1) ...
Setting up bedtools (2.30.0+dfsg-2ubuntu0.1) ...


# Получим файлы промоторов/экзонов/межгенников/интронов и downstream

In [24]:
!wget -r -nH --cut-dirs=5 ftp.ncbi.nlm.nih.gov/genomes/all/GCA/036/169/595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.fna.gz

URL transformed to HTTPS due to an HSTS policy
--2025-06-15 13:17:02--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/036/169/595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.31, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9089934 (8.7M) [application/x-gzip]
Saving to: ‘595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.fna.gz’

595/GCA_036169595.1 100%[===================>]   8.67M  15.3MB/s    in 0.6s    

2025-06-15 13:17:03 (15.3 MB/s) - ‘595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.fna.gz’ saved [9089934/9089934]

FINISHED --2025-06-15 13:17:03--
Total wall clock time: 1.1s
Downloaded: 1 files, 8.7M in 0.6s (15.3 MB/s)


In [28]:
!gzip -rd GCA_036169595.1_ASM3616959v1_genomic.fna.gz

In [26]:
!wget -r -nH --cut-dirs=5 ftp.ncbi.nlm.nih.gov/genomes/all/GCA/036/169/595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.gtf.gz

URL transformed to HTTPS due to an HSTS policy
--2025-06-15 13:17:10--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/036/169/595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.gtf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.31, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1742757 (1.7M) [application/x-gzip]
Saving to: ‘595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.gtf.gz’

595/GCA_036169595.1 100%[===================>]   1.66M  3.81MB/s    in 0.4s    

2025-06-15 13:17:11 (3.81 MB/s) - ‘595/GCA_036169595.1_ASM3616959v1/GCA_036169595.1_ASM3616959v1_genomic.gtf.gz’ saved [1742757/1742757]

FINISHED --2025-06-15 13:17:11--
Total wall clock time: 0.7s
Downloaded: 1 files, 1.7M in 0.4s (3.81 MB/s)


In [27]:
!gzip -rd GCA_036169595.1_ASM3616959v1_genomic.gtf.gz

In [29]:
def calculate_scaffold_lengths(fasta_file, output_file):
    with open(output_file, 'w') as out:
        for record in SeqIO.parse(fasta_file, "fasta"):
            scaffold_name = record.id
            scaffold_length = len(record.seq)
            out.write(f"{scaffold_name}\t{scaffold_length}\n")
calculate_scaffold_lengths('GCA_036169595.1_ASM3616959v1_genomic.fna', 'scaffolds.sizes')

In [31]:
!awk '{if($1 !~ /^#/ ) print $1"\t"$4-1"\t"$5"\t"$3"\t"$10"\t"$7}' GCA_036169595.1_ASM3616959v1_genomic.gtf > annotation.bed

In [32]:
!head annotation.bed

CM069765.1	3773	3847	gene	"RB653_010779";	+
CM069765.1	3773	3847	transcript	"RB653_010779";	+
CM069765.1	3773	3847	exon	"RB653_010779";	+
CM069765.1	5009	8310	gene	"RB653_003324";	+
CM069765.1	5009	8310	transcript	"RB653_003324";	+
CM069765.1	5009	5093	exon	"RB653_003324";	+
CM069765.1	5191	5424	exon	"RB653_003324";	+
CM069765.1	5512	5796	exon	"RB653_003324";	+
CM069765.1	5871	6633	exon	"RB653_003324";	+
CM069765.1	6694	8310	exon	"RB653_003324";	+


In [33]:
!awk '$4 == "exon"' annotation.bed > exons.bed

In [34]:
!head exons.bed

CM069765.1	3773	3847	exon	"RB653_010779";	+
CM069765.1	5009	5093	exon	"RB653_003324";	+
CM069765.1	5191	5424	exon	"RB653_003324";	+
CM069765.1	5512	5796	exon	"RB653_003324";	+
CM069765.1	5871	6633	exon	"RB653_003324";	+
CM069765.1	6694	8310	exon	"RB653_003324";	+
CM069765.1	8574	10467	exon	"RB653_003325";	+
CM069765.1	10788	10823	exon	"RB653_003326";	+
CM069765.1	10889	11699	exon	"RB653_003326";	+
CM069765.1	11779	12401	exon	"RB653_003326";	+


In [35]:
!awk 'BEGIN{OFS="\t"} {if ($4 == "gene") {print $1, $2, $3, $4, $5, $6}}' annotation.bed > genes.bed

In [36]:
!bedtools flank -i genes.bed -g scaffolds.sizes -l 1000 -r 0 -s > promoters.bed

In [37]:
!head promoters.bed

CM069765.1	2773	3773	gene	"RB653_010779";	+
CM069765.1	4009	5009	gene	"RB653_003324";	+
CM069765.1	7574	8574	gene	"RB653_003325";	+
CM069765.1	9788	10788	gene	"RB653_003326";	+
CM069765.1	18014	19014	gene	"RB653_003336";	-
CM069765.1	18644	19644	gene	"RB653_010780";	-
CM069765.1	19053	20053	gene	"RB653_003327";	+
CM069765.1	21801	22801	gene	"RB653_003337";	-
CM069765.1	21135	22135	gene	"RB653_003328";	+
CM069765.1	27853	28853	gene	"RB653_003338";	-


In [38]:
!awk '$4=="transcript"' annotation.bed > transcripts.bed

In [39]:
!head transcripts.bed

CM069765.1	3773	3847	transcript	"RB653_010779";	+
CM069765.1	5009	8310	transcript	"RB653_003324";	+
CM069765.1	8574	10467	transcript	"RB653_003325";	+
CM069765.1	10788	12401	transcript	"RB653_003326";	+
CM069765.1	13577	18014	transcript	"RB653_003336";	-
CM069765.1	18570	18644	transcript	"RB653_010780";	-
CM069765.1	20053	20407	transcript	"RB653_003327";	+
CM069765.1	20486	21801	transcript	"RB653_003337";	-
CM069765.1	22135	27167	transcript	"RB653_003328";	+
CM069765.1	26685	27853	transcript	"RB653_003338";	-


In [40]:
!bedtools subtract -a transcripts.bed -b exons.bed > introns.bed

In [41]:
!head introns.bed

CM069765.1	5093	5191	transcript	"RB653_003324";	+
CM069765.1	5424	5512	transcript	"RB653_003324";	+
CM069765.1	5796	5871	transcript	"RB653_003324";	+
CM069765.1	6633	6694	transcript	"RB653_003324";	+
CM069765.1	10823	10889	transcript	"RB653_003326";	+
CM069765.1	11699	11779	transcript	"RB653_003326";	+
CM069765.1	16942	17034	transcript	"RB653_003336";	-
CM069765.1	17506	17576	transcript	"RB653_003336";	-
CM069765.1	20712	20784	transcript	"RB653_003337";	-
CM069765.1	21079	21230	transcript	"RB653_003337";	-


In [42]:
!bedtools complement -i genes.bed -g scaffolds.sizes > intergenic.bed

In [43]:
!head intergenic.bed

CM069765.1	0	3773
CM069765.1	3847	5009
CM069765.1	8310	8574
CM069765.1	10467	10788
CM069765.1	12401	13577
CM069765.1	18014	18570
CM069765.1	18644	20053
CM069765.1	20407	20486
CM069765.1	21801	22135
CM069765.1	27853	28147


In [44]:
!bedtools flank -i genes.bed -g scaffolds.sizes -l 0 -r 200 -s  > downstream.bed

In [45]:
!head downstream.bed

CM069765.1	3847	4047	gene	"RB653_010779";	+
CM069765.1	8310	8510	gene	"RB653_003324";	+
CM069765.1	10467	10667	gene	"RB653_003325";	+
CM069765.1	12401	12601	gene	"RB653_003326";	+
CM069765.1	13377	13577	gene	"RB653_003336";	-
CM069765.1	18370	18570	gene	"RB653_010780";	-
CM069765.1	20407	20607	gene	"RB653_003327";	+
CM069765.1	20286	20486	gene	"RB653_003337";	-
CM069765.1	27167	27367	gene	"RB653_003328";	+
CM069765.1	26485	26685	gene	"RB653_003338";	-


# Данные по zdnabert

In [46]:
!bedtools intersect -a ZDNABERT.bed -b exons.bed -wa > ZDNABERT_in_exons.bed

In [47]:
!bedtools intersect -a ZDNABERT.bed -b introns.bed -wa > ZDNABERT_in_introns.bed

In [48]:
!bedtools intersect -a ZDNABERT.bed -b promoters.bed -wa > ZDNABERT_in_promoters.bed

In [49]:
!bedtools intersect -a ZDNABERT.bed -b intergenic.bed -wa > ZDNABERT_in_intergenic.bed

In [50]:
!bedtools intersect -a ZDNABERT.bed -b downstream.bed -wa > ZDNABERT_in_downstream.bed

In [51]:
!cat ZDNABERT.bed | wc -l

7


In [52]:
!cat ZDNABERT_in_exons.bed | wc -l

3


In [53]:
!cat ZDNABERT_in_introns.bed | wc -l

0


In [54]:
!cat ZDNABERT_in_promoters.bed | wc -l

2


In [55]:
!cat ZDNABERT_in_intergenic.bed | wc -l

4


In [56]:
!cat ZDNABERT_in_downstream.bed | wc -l

1
